In [274]:
import random
import string
    

DEFINIMOS la funcion que traduce de **texto** a **numeros**

_NOTA_ => Se traduce siempre el texto de entrada a mayusculas


In [275]:
def textoAnumeros(texto):
    texto_min = texto.upper()
    lista =[]
    for letra in texto_min:
        lista.append(string.ascii_uppercase.index(letra)+1)
    return lista

DEFINIMOS la funcion que traduce de **numeros** a **texto**

In [276]:
def numerosAtexto(numeros):
    textoFinal= ""
    for num in numeros:
        textoFinal= textoFinal + string.ascii_uppercase[num-1]
    return textoFinal

DEFINIMOS la funcion que **mueve** los **jokers** por la baraja

Recibe como argumentos de entrada:
    * Joker a mover (A=53 o B=54)    
    * Baraja de cartas
 Devuelve:
    * Baraja con el joker reposicionado

In [277]:
def moverJokers(joker, baraja):
    
    # Vemos el numero de movimientos en la baraja, en base al joker que sea
    if (joker==53): 
        posMover = 1
    else:
        posMover = 2
        
    # Buscamos su posicion actual en la baraja
    indice = baraja.index(joker)
    
    #Si esta al final de la baraja, es como si estuviese al principio
    if (indice == len(baraja)-1): 
        nuevoIndice = posMover
    # Si se trata del joker B y esta en la anteultima posicion
    elif ( (indice == len(baraja)-2) and (joker == 54) ): 
        nuevoIndice = 1
    #resto de casos se calculan con normalidad, su posicion mas el numero de elementos que corresponda
    else: 
        nuevoIndice = indice + posMover
    
    baraja.insert(nuevoIndice, baraja.pop(indice))
    
    return baraja

DEFINIMOS la función que hace el triple corte en la baraja

Recibe como argumento de ENTRADA:
    * la baraja
como SALIDA devuelve:
    * la misma baraja pero tras realizarle el llamado triple corte

In [278]:
def tripleCorte(baraja):
    nuevaBaraja=[]
    posJokerA = baraja.index(53)
    posJokerB = baraja.index(54)
    
    #Miramos cual esta en primera posicion
    if ( posJokerA < posJokerB ):
        primerCorte = posJokerA
        segundoCorte = posJokerB
    else:
        primerCorte = posJokerB
        segundoCorte = posJokerA
    
    primeraParte = baraja[0:primerCorte]
    segundaParte = baraja[primerCorte:segundoCorte+1]
    terceraParte = baraja[segundoCorte+1:]
    
    # Generamos la nueva baraja, con el primer y tercer corte intercambiados
    nuevaBaraja.extend(terceraParte)
    nuevaBaraja.extend(segundaParte)
    nuevaBaraja.extend(primeraParte)
    
    return nuevaBaraja


DEFINIMOS la función que realiza el corte de conteo

Recibe como argumento de ENTRADA:
    * la baraja
como SALIDA devuelve
    * la baraja tras realizarle el corte de conteo

In [279]:
def corteDeConteo(baraja):
    nuevaBaraja=[]
    ultimoValor = baraja[len(baraja) -1]
    
    #Si la ultima carta no es ninguno de los 2 jokers
    if (ultimoValor!=53 and ultimoValor!= 54):
        primeraParte =baraja[:ultimoValor]
        segundaParte =baraja[ultimoValor:len(baraja) -1]

        nuevaBaraja.extend(segundaParte)
        nuevaBaraja.extend(primeraParte)
        nuevaBaraja.append(ultimoValor)
    # Si la ultima carta es alguno de los 2 jokers, devolvemos la baraja tal cual
    else:
        nuevaBaraja = baraja
        
    return nuevaBaraja


DEFINIMOS la funcion que calcula para cada posicion cual es el keystream que le corresponde

In [280]:
def keystream(baraja):
    valorPrimeraCarta = baraja[0]
    if (valorPrimeraCarta==53 or valorPrimeraCarta==54):
        valor = baraja[len(baraja)-1]
    else:
        valor = baraja[valorPrimeraCarta]
    
    
    return valor

#baraja=[54, 4, 5, 6, 53, 54, 2]
#valor = keystream(baraja)
#print(valor)

DEFINIMOS la funcion que llama a todas las anteriores y devuelve el keystream completo, para el texto que se le pasa

In [281]:
def generarKeystream(texto, baraja):
    keystreamCompleto =[]
    while (len(keystreamCompleto) < len(texto)):
        baraja = moverJokers(53, baraja)
        baraja = moverJokers(54, baraja)
        baraja = tripleCorte(baraja)
        baraja = corteDeConteo(baraja)
        valor = keystream(baraja)
        if (valor != ''):
            keystreamCompleto.append(valor)
    
    return keystreamCompleto


DEFINIMOS la funcion que sumara el texto original en numero, con el keystream generado para obtener el texto cifradado en numeros

In [282]:
def sumarParaEncriptar(textoEntNumerico, textoKeystream):
    textoCrifradoNumerico=[]
    for i in range(len(textoEntNumerico)):
        valorSumado = textoEntNumerico[i] + textoKeystream[i]
        if valorSumado > 52:
            valorFinal = valorSumado - 52
        elif valorSumado > 26:
            valorFinal = valorSumado - 26
        else:
            valorFinal = valorSumado
        
        textoCrifradoNumerico.append(valorFinal)
        
    return textoCrifradoNumerico

DEFINIMOS la funcion que restara el texto cifrado en numero, el keystream generado para obtener el texto descifradado en numeros

In [283]:
def restarParaDesencriptar(textoSalNumerico, textoSalKeystream):
    textoS_CifradoNumerico = []
    for i in range(len(textoSalNumerico)):
        valorRestado = textoSalNumerico[i] - textoSalKeystream[i] 
        if valorRestado < -26:
            valorFinal = (valorRestado + 52)
        elif valorRestado < 0:
            valorFinal = (valorRestado + 26)
        else:
            valorFinal = valorRestado
        
        textoS_CifradoNumerico.append(valorFinal)
        
    return textoS_CifradoNumerico

DEFINIMOS la fucnion que valida que el texto de entrada no contiene numeros

In [284]:
def validarTexto(textoEntrada):
    for letra in textoEntrada:
        if letra.isdigit():
            return False
            break
    return True

DEFINIMOS el algoritmo completo, al que debemos llamar para ejecutar el cifrado y descifrado de un numero con el solitario

In [285]:
def algoritmoCompleto(textoEntrada):
    
    # Quitamos los posibles espacios en balnco
    textoEntrada= textoEntrada.replace(" ", "")

    # 1º Creamos un array de valores aleatorios que representarán la baraja de cartas
    barajaOriginal = []
    for i in range(1,55):
        barajaOriginal.insert(i, i)
    # la barajamos y creamos una copia, que es la que se enviará al descifrado
    random.shuffle(barajaOriginal)
    baraja2 = barajaOriginal.copy()

    # 2º Creamos el alfabeto con el que traduciremos el texto a numero y viceversa
    alfabeto = list(string.ascii_uppercase)

    # 3º Ciframos el mensaje
    # Cambiamos el texto de letras a posiciones en el alfabeto
    textoEntNumerico = textoAnumeros(textoEntrada)
    textoKeystream= generarKeystream(textoEntrada, barajaOriginal)
    numerosCifrados = sumarParaEncriptar(textoEntNumerico, textoKeystream)
    textoEntCifrado = numerosAtexto(numerosCifrados)

    # 4º => Desciframos el mensaje
    # ahora enviamos el mensaje cifrado y lo desciframos
    textoSalNumerico = textoAnumeros(textoEntCifrado)
    textoSalKeystream= generarKeystream(textoSalNumerico, baraja2)
    numerosSalCifrados = restarParaDesencriptar(textoSalNumerico, textoSalKeystream)
    textoSalCifrado = numerosAtexto(numerosSalCifrados)

    """
    # Pintamos todos los datos para poder comprobar los pasos
    print("DATOS PARA ENCRIPTAR")
    print(barajaOriginal)
    print('textoEntrada = ' + textoEntrada)
    print('textoEntNumerico = ')
    print(textoEntNumerico)
    print('textoKeystream = ')
    print(textoKeystream)
    print('numerosCifrados = ')
    print(numerosCifrados)
    print('textoEntCifrado = ')
    print(textoEntCifrado)
    print('\n')
    print("DATOS PARA DESENCRIPTAR")
    print(baraja2)
    print('textoSalNumerico = ')
    print(textoSalNumerico)
    print('textoSalKeystream = ')
    print(textoSalKeystream)
    print('numerosSalCifrados = ')
    print(numerosSalCifrados)
    print('textoSalCifrado = ')
    print(textoSalCifrado)
    """
    return textoEntCifrado, textoSalCifrado


DESDE aqui es donde puedes introducir tu texto para llamar a la fucnion de **encriptado** y **desencriptado** con **el solitario**

__NOTAS__
    * Si introduces algun numero en el texto, no te dejaré probar la funcion
    * si tu texto tiene algun caracter en blanco, lo eliminare y considerare tu como texto de entrada tu texto sin espacios

In [286]:
textoEntrada = "VALIDADO"
if validarTexto(textoEntrada):
    textoCifrado, textDescrifrado = algoritmoCompleto(textoEntrada)
    print("El texto de entrada es " + textoEntrada)
    print("El texto cifrado es " + textoCifrado)
    print("El texto descifrado es " + textDescrifrado)
else:
    print("Tu texto contine números, no lo podemos cifrar con esta función")

El texto de entrada es VALIDADO
El texto cifrado es XCLSGNLC
El texto descifrado es VALIDADO
